# XGBoost

### 1. 필요한 Python 라이브러리 Import 하기

In [1]:
# pandas와 numpy 임포트하기
import pandas as pd
import numpy as np

# 경고 끄기
import warnings
warnings.filterwarnings("ignore")

import xgboost as xgb
from sklearn.metrics import mean_squared_error as MSE

# XGBRegressor를 임포트합니다.
from xgboost import XGBRegressor

xgb.set_config(verbosity=0)

In [2]:
df_bikes = pd.read_csv("bike_rentals_cleaned.csv")
df_bikes.head()

,instant,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,cnt
0,1,1.0,0.0,1.0,0.0,6.0,0.0,2,0.344167,0.363625,0.805833,0.160446,985
1,2,1.0,0.0,1.0,0.0,0.0,0.0,2,0.363478,0.353739,0.696087,0.248539,801
2,3,1.0,0.0,1.0,0.0,1.0,1.0,1,0.196364,0.189405,0.437273,0.248309,1349
3,4,1.0,0.0,1.0,0.0,2.0,1.0,1,0.200000,0.212122,0.590435,0.160296,1562
4,5,1.0,0.0,1.0,0.0,3.0,1.0,1,0.226957,0.229270,0.436957,0.186900,1600


In [3]:
# 데이터를 X와 y로 나눕니다.
X_bikes = df_bikes.iloc[:,:-1]
y_bikes = df_bikes.iloc[:,-1]

# train_test_split를 임포트합니다.
from sklearn.model_selection import train_test_split

# 데이터를 훈련 세트와 테스트 세트로 나눕니다.
X_train, X_test, y_train, y_test = train_test_split(X_bikes, y_bikes, random_state=2)

### 2. 기본 XGBoost

In [4]:
# XGBRegressor를 임포트합니다.
from xgboost import XGBRegressor

# XGBRegressor를 초기화합니다.
xg_reg = XGBRegressor(
    max_depth=3, n_estimators=1600, eta=0.02, subsample=0.75, random_state=2
)

# 훈련 세트에서 xg_reg를 훈련합니다.
xg_reg.fit(X_train, y_train)

# 테스트 세트에 대한 예측을 만듭니다.
y_pred = xg_reg.predict(X_test)

# 평균 제곱근 오차(RMSE) 계산합니다.
MSE(y_test, y_pred) ** 0.5

588.1965020297214

### 3. cross_val_score 

In [5]:
# cross_val_score를 임포트합니다.
from sklearn.model_selection import cross_val_score

# numpy를 임포트합니다.
import numpy as np

model = XGBRegressor(booster='gbtree', objective='reg:squarederror')

# 교차 검증 점수를 구합니다.
scores = cross_val_score(model, X_bikes, y_bikes, scoring='neg_mean_squared_error', cv=5)

# 정확도를 출력합니다.
rmse = np.sqrt(-scores)
print('RMSE:', np.round(rmse, 2))

# 정확도 평균을 출력합니다.
print('RMSE 평균: %0.2f' % (rmse.mean()))

RMSE: [1103.54  614.75 1275.99  901.21 1171.78]
RMSE 평균: 1013.45


### 4. XGBoost 하이퍼파라미터 튜닝

In [6]:
# RandomizedSearchCV 임포트
from sklearn.model_selection import RandomizedSearchCV


def randomized_search_reg(
    params, runs=16, reg=XGBRegressor(random_state=2, n_jobs=-1)
):

    # RandomizedSearchCV 객체를 만듭니다.
    rand_reg = RandomizedSearchCV(
        reg,
        params,
        n_iter=runs,
        scoring="neg_mean_squared_error",
        cv=10,
        n_jobs=-1,
        random_state=2,
    )

    # X_train와 y_train로 grid_reg를 훈련합니다.
    rand_reg.fit(X_train, y_train)

    # 최상의 모델을 추출합니다.
    best_model = rand_reg.best_estimator_

    # 최상의 매개변수를 추출합니다.
    best_params = rand_reg.best_params_

    # 최상의 매개변수를 출력합니다.
    print("최상의 매개변수:", best_params)

    # 최상의 점수를 구합니다.
    best_score = np.sqrt(-rand_reg.best_score_)

    # 최상의 점수를 출력합니다.
    print("훈련 점수: {:.3f}".format(best_score))

    # 테스트 세트의 레이블을 예측합니다.
    y_pred = best_model.predict(X_test)

    # mean_squared_error를 임포트합니다.
    from sklearn.metrics import mean_squared_error as MSE

    # rmse를 계산합니다.
    rmse_test = MSE(y_test, y_pred) ** 0.5

    # rmse를 출력합니다.
    print("테스트 세트 점수: {:.3f}".format(rmse_test))

#### n_estimators

In [7]:
randomized_search_reg(params={'n_estimators':[100, 200, 400, 800, 1200, 1600]})

최상의 매개변수: {'n_estimators': 100}
훈련 점수: 682.120
테스트 세트 점수: 693.210


#### learning_rate

In [8]:
randomized_search_reg(params={'learning_rate':[0.01, 0.02, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5]})

최상의 매개변수: {'learning_rate': 0.1}
훈련 점수: 671.427
테스트 세트 점수: 667.668


#### max_depth

In [9]:
randomized_search_reg(params={'max_depth':[2, 3, 5, 6, 8]})

최상의 매개변수: {'max_depth': 2}
훈련 점수: 642.346
테스트 세트 점수: 652.164


#### gamma

In [10]:
randomized_search_reg(params={'gamma':[0, 0.01, 0.1, 0.5, 1, 2]})

최상의 매개변수: {'gamma': 0.5}
훈련 점수: 682.048
테스트 세트 점수: 693.448


#### min_child_weight

In [11]:
randomized_search_reg(params={'min_child_weight':[1, 2, 3, 4, 5]})

최상의 매개변수: {'min_child_weight': 3}
훈련 점수: 674.861
테스트 세트 점수: 679.417


#### subsample

In [12]:
randomized_search_reg(params={'subsample':[0.5, 0.7, 0.8, 0.9, 1]})

최상의 매개변수: {'subsample': 0.7}
훈련 점수: 675.532
테스트 세트 점수: 637.274


#### colsample_bytree

In [13]:
randomized_search_reg(params={'colsample_bytree':[0.5, 0.7, 0.8, 0.9, 1]})

최상의 매개변수: {'colsample_bytree': 0.8}
훈련 점수: 667.598
테스트 세트 점수: 660.784


#### n_estimators

In [14]:
randomized_search_reg(params={'n_estimators':[2, 25, 50, 75, 100]})

최상의 매개변수: {'n_estimators': 50}
훈련 점수: 681.051
테스트 세트 점수: 696.109


#### max_depth and n_estimators (1)

In [15]:
randomized_search_reg(params={'max_depth':[1, 2, 3, 4, 6, 7, 8], 
                    'n_estimators':[50]})

최상의 매개변수: {'n_estimators': 50, 'max_depth': 4}
훈련 점수: 650.526
테스트 세트 점수: 633.794


#### max_depth and n_estimators (2)

In [16]:
randomized_search_reg(params={'max_depth':[1, 2, 3, 4, 6, 7, 8], 
                    'n_estimators':[2, 50, 100]})

최상의 매개변수: {'n_estimators': 100, 'max_depth': 2}
훈련 점수: 642.346
테스트 세트 점수: 652.164


#### learning_rate, max_depth and n_estimators

In [17]:
randomized_search_reg(params={'learning_rate':[0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5], 
                    'max_depth':[1], 'n_estimators':[50]})

최상의 매개변수: {'n_estimators': 50, 'max_depth': 1, 'learning_rate': 0.4}
훈련 점수: 692.755
테스트 세트 점수: 735.912


#### min_child_weight, max_depth and n_estimators

In [18]:
randomized_search_reg(params={'min_child_weight':[1, 2, 3, 4, 5], 
                    'max_depth':[1], 'n_estimators':[50]})

최상의 매개변수: {'n_estimators': 50, 'min_child_weight': 1, 'max_depth': 1}
훈련 점수: 696.721
테스트 세트 점수: 741.719


#### subsample, max_depth and n_estimators

In [19]:
randomized_search_reg(params={'subsample':[0.5, 0.6, 0.7, 0.8, 0.9, 1],
                    'max_depth':[1], 'n_estimators':[50]})

최상의 매개변수: {'subsample': 0.6, 'n_estimators': 50, 'max_depth': 1}
훈련 점수: 694.462
테스트 세트 점수: 738.485


#### subsample, min_child_weight, learning_rate, max_depth and n_estimators (1)

In [20]:
randomized_search_reg(params={'subsample':[0.5, 0.6, 0.7, 0.8, 0.9, 1], 
                    'min_child_weight':[1, 2, 3, 4, 5], 
                    'learning_rate':[0.1, 0.2, 0.3, 0.4, 0.5], 
                    'max_depth':[1, 2, 3, 4, 5], 
                    'n_estimators':[2]})

최상의 매개변수: {'subsample': 0.5, 'n_estimators': 2, 'min_child_weight': 2, 'max_depth': 5, 'learning_rate': 0.5}
훈련 점수: 952.409
테스트 세트 점수: 948.541


#### subsample, min_child_weight, learning_rate, max_depth and n_estimators (2) 

In [21]:
randomized_search_reg(params={'subsample':[0.5, 0.6, 0.7, 0.8, 0.9, 1], 
                    'min_child_weight':[1, 2, 3, 4, 5], 
                    'learning_rate':[0.1, 0.2, 0.3, 0.4, 0.5], 
                    'max_depth':[1, 2, 3, 4, 5, None], 
                    'n_estimators':[2, 25, 50, 75, 100]})

최상의 매개변수: {'subsample': 0.6, 'n_estimators': 75, 'min_child_weight': 2, 'max_depth': 2, 'learning_rate': 0.2}
훈련 점수: 635.924
테스트 세트 점수: 669.161


#### colsample_bytree, max_depth and n_estimators

In [22]:
randomized_search_reg(params={'colsample_bytree':[0.5, 0.6, 0.7, 0.8, 0.9, 1],
                    'max_depth':[1], 'n_estimators':[50]})

최상의 매개변수: {'n_estimators': 50, 'max_depth': 1, 'colsample_bytree': 0.6}
훈련 점수: 694.858
테스트 세트 점수: 736.901


#### colsample_bylevel, max_depth and n_estimators

In [23]:
randomized_search_reg(params={'colsample_bylevel':[0.5, 0.6, 0.7, 0.8, 0.9, 1],
                    'max_depth':[1], 'n_estimators':[50]})

최상의 매개변수: {'n_estimators': 50, 'max_depth': 1, 'colsample_bylevel': 0.9}
훈련 점수: 695.113
테스트 세트 점수: 741.320


#### gamma, colsample_bylevel, colsample_bytree, colsample_bynode, max_depth and n_estimators

In [24]:
randomized_search_reg(params={'gamma':[0, 0.01, 0.05, 0.1, 0.5, 1, 2, 3], 
                    'colsample_bylevel':[0.9], 
                    'colsample_bytree':[0.8], 
                    'colsample_bynode':[0.5], 
                    'max_depth':[1], 'n_estimators':[50]})

최상의 매개변수: {'n_estimators': 50, 'max_depth': 1, 'gamma': 0, 'colsample_bytree': 0.8, 'colsample_bynode': 0.5, 'colsample_bylevel': 0.9}
훈련 점수: 703.807
테스트 세트 점수: 745.474


#### Best Model

In [25]:
randomized_search_reg(params={'max_depth':[3], 
                               'n_estimators':[1600],
                               'eta':[0.02],
                               'subsample':[0.75],
                               'random_state':[2],
                               'min_child_weight':[1, 2, 3, 4, 5]})

최상의 매개변수: {'subsample': 0.75, 'random_state': 2, 'n_estimators': 1600, 'min_child_weight': 5, 'max_depth': 3, 'eta': 0.02}
훈련 점수: 615.035
테스트 세트 점수: 584.246
